In [2]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
from demo_model import SGG_Model
import cv2
import time

config_path = "/home/maelic/ros2_humble/src/Robots-Scene-Understanding/rsu_scene_graph_generation/models/penet-yolov8m/config.yaml"
dict_path = "../datasets/IndoorVG_4/VG-SGG-dicts.json"

example_img = "./test_custom/example.jpg"
# load image with cv2
img = cv2.imread(example_img)

model = SGG_Model(config_path, dict_path)
img_list, target = model._pre_processing(img)
targets = [target.to(model.device)]
img_list = img_list.to(model.device)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        time_start = time.time()
        with torch.no_grad():

            outputs, features = model.model.backbone(img_list.tensors, embed=True)
            proposals = model.model.backbone.postprocess(outputs, img_list.image_sizes)

            _, predictions, _ = model.model.roi_heads(features, proposals, targets, None, proposals)

        time_end = time.time()
print(f"Time: {time_end - time_start}")
#predictions = model._post_process_yolo(predictions[0], orig_size=img_list.image_sizes[0])

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

total_params = sum(p.numel() for p in model.model.parameters())
print(f"Number of parameters: {total_params}")

2024-04-09 18:50:36.405 | INFO     | sgg_benchmark.utils.logger:setup_logger:30 - Using loguru logger with level: INFO
Overriding model.yaml nc=80 with nc=84

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             


  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384, 576, 3, 2]              
  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

STAGE:2024-04-09 18:50:39 947026:947026 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-09 18:50:39 947026:947026 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-09 18:50:39 947026:947026 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


Time: 0.028138160705566406
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        20.04%       6.387ms        88.56%      28.228ms      28.228ms       0.000us         0.00%      22.143ms      22.143ms             1  
                                           aten::conv2d         1.30%     415.000us        14.50%       4.620ms      51.910us       0.000us         0.00%       9.594ms     107.798u

In [15]:
total_params = sum(p.numel() for p in model.model.parameters())
print(f"Number of parameters of full model: {total_params}")

total_params = sum(p.numel() for p in model.model.backbone.parameters())
print(f"Number of parameters of backbone: {total_params}")

total_params = sum(p.numel() for p in model.model.roi_heads.parameters())
print(f"Number of parameters of Relation head: {total_params}")

Number of parameters of full model: 171628072
Number of parameters of backbone: 25904956
Number of parameters of Relation head: 145723116


In [ ]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
from demo_model import SGG_Model
import cv2
import time

config_path = "./checkpoints/IndoorVG4/SGDET/penet-faster_rcnn/config.yml"
dict_path = "../datasets/IndoorVG/VG-SGG-dicts.json"

example_img = "./test_custom/example.jpg"
# load image with cv2
img = cv2.imread(example_img)

model = SGG_Model(config_path, dict_path)
img_list, target = model._pre_processing(img)
target = torch.LongTensor([-1])
targets = [target.to(model.device)]
img_list = img_list.to(model.device)
model.model.eval()

# warm-up

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        time_start = time.time()
        with torch.no_grad():
            torch.cuda.empty_cache()
            # for i in range(10):
            features = model.model.backbone(img_list.tensors)
            proposals, proposal_losses = model.model.rpn(img_list, features, targets)
            x, result, detector_losses = model.model.roi_heads(features, proposals, targets)
            # del x, result, detector_losses
            torch.cuda.empty_cache()
            print(time.time() - time_start)
        time_end = time.time()
print(f"Time: {(time_end - time_start)/10}")
#predictions = model._post_process_yolo(predictions[0], orig_size=img_list.image_sizes[0])

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

total_params = sum(p.numel() for p in model.model.parameters())
print(f"Number of parameters: {total_params}")